In [ ]:
# %load_ext autoreload
# %autoreload 2
%load_ext lab_black
import sys

sys.path.append("~/div/wallet/")

In [ ]:
from wallet.wallet import Wallet, Account, headers, Transaction, IGNORE_ACCOUNTS
import patito as pt
import pandas as pd
import polars as pl
from datetime import datetime

In [ ]:
accounts = Wallet.accounts()

In [ ]:
%pip install dotenv

In [ ]:
transactions = Wallet.transactions(accounts[2], fromDate="2022-02-02")
df = Transaction.DataFrame(pd.DataFrame(transactions))

In [ ]:
def get_transfer_account(number: int):
    acct = Wallet.account_from_number(number)
    if acct:
        if acct.name in IGNORE_ACCOUNTS:
            return f"Payment t/f {acct.name}"
        return f"Transfer t/f {acct.name}"


def clean_data(df: pt.DataFrame):
    if df.is_empty():
        return df
    return (
        df.drop()
        .cast()
        .with_columns(
            [
                (pl.col("date") // 1000)
                .apply(datetime.fromtimestamp)
                .apply(datetime.date),
                pl.when(
                    pl.col("remoteAccountNumber")
                    .apply(get_transfer_account)
                    .is_not_null()
                )
                .then(pl.col("remoteAccountNumber").apply(get_transfer_account))
                .otherwise(pl.col("description"))
                .alias("description"),
            ]
        )
        # .drop("remoteAccountNumber")
    )

In [ ]:
clean_data(df)